- Train a simple baeline with Transormers & Datasets

In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install accelerate -U
!pip install sentencepiece

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [ ]:
model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-5
epoch = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
train_dataset = load_dataset("SajjadAyoubi/persian_qa")
train_dataset[0]

Repo card metadata block was not found. Setting CardData to empty.


{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [ ]:
val_dataset = load_dataset("SajjadAyoubi/persian_qa")
val_dataset[0]

Repo card metadata block was not found. Setting CardData to empty.


{'id': 9101,
 'title': 'باشگاه فوتبال رئال مادرید',
 'context': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق\u200cترین تیم تاریخ فوتبال اسپانیا و موفق\u200cترین تیم تاریخ فوتبال اروپا و موفق\u200cترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن\u200cها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه\u200cهای جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده\u200cاست. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال\u200cهاست که در لالیگا، از تیم\u200cهای قدرتمند محسوب می\u200cشود و رقیب اصلی این تیم نیز، بارسل

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
token_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

In [ ]:
token_val = val_dataset.map(prepare_train_features, batched=True)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
!huggingface-cli login --token "YOUR_TOKEN"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
args = TrainingArguments(
    f"qa-persian",
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    num_train_epochs=epoch,
    weight_decay=0.0001)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=token_train,
    eval_dataset=token_val,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

In [ ]:
trainer.create_model_card()

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/AliBagherz/qa-persian/tree/main/'

In [ ]:
tokenizer.push_to_hub("AliBagherz/qa-persian")

CommitInfo(commit_url='https://huggingface.co/AliBagherz/qa-persian/commit/8e2f2948a209ef29ad4909a5bd8aa8bc00e1ff43', commit_message='Upload tokenizer', commit_description='', oid='8e2f2948a209ef29ad4909a5bd8aa8bc00e1ff43', pr_url=None, pr_revision=None, pr_num=None)